In [11]:
import pandas as pd
import requests
# API parameters
cycle = 2024  # Specify the election cycle
office = 'house'  # Specify the office type
api_key = 'BVTfalQokoMzRLDe1oCsI1k9T4OxyttZ56GQFOhZ'  # Replace with your actual API key

# Base URL for the OpenFEC API
base_url = "https://api.open.fec.gov/v1/elections/"

# List of all US state abbreviations
toss_up_seats_combined_updated = [
    ("AK", "00"), ("CO", "08"), ("ME", "02"),
    ("MI", "07"), ("MI", "08"), ("NC", "01"), 
    ("NM", "02"), ("PA", "07"), ("PA", "08"), 
    ("VA", "07"), ("WA", "03"), ("AZ", "01"), 
    ("AZ", "06"), ("CA", "13"), ("CA", "22"), 
    ("CA", "27"), ("CA", "41"), ("CA", "45"), 
    ("IA", "01"), ("IA", "03"), ("NE", "02"), 
    ("NY", "04"), ("NY", "19"), ("OR", "05"), 
    ("PA", "10"), ("CA", "47"), ("CA", "49"), 
    ("CT", "05"), ("MN", "02"), ("NV", "03"), 
    ("NY", "18"), ("NY", "22"), ("OH", "09"), 
    ("OH", "13"), ("OR", "06"), ("PA", "17"), 
    ("TX", "34")
]


# Dictionary to store DataFrames for each state
seat_data = {}

for seat in seats:
    # Make the API call
    response = requests.get(f"{base_url}?cycle={cycle}&office={office}&state={seat[0]}&district={seat[1]}&election_full=true&api_key={api_key}")

    # Check if the request was successful
    if response.status_code == 200:

        # Parse the JSON response
        data = response.json()

        
        df = pd.DataFrame(data['results'])
            # Store the DataFrame in the dictionary
        seat_data[seat] = df
    else:           
        print(f"Failed to retrieve data for {seat}: {response.status_code} - {response.text}")

print(seat_data)

{('AK', '00'):    candidate_election_year candidate_id             candidate_name  \
0                     2024    H2AK01158              PELTOLA, MARY   
1                     2024    H2AK01083       BEGICH, NICHOLAS III   
2                     2024    H4AK00156           DAHLSTROM, NANCY   
3                     2024    H2AK00556             AMBROSE, DAVID   
4                     2024    H2AK01182       DUTCHESS, LADY DONNA   
5                     2024    H4AK00172      SALISBURY, MATTHEW H.   
6                     2024    H4AK00214           HOWE, JOHN WAYNE   
7                     2024    H4AK01030  SALISBURY, MATTHEW HARRIS   

  candidate_pcc_id              candidate_pcc_name  cash_on_hand_end_period  \
0        C00812388         MARY PELTOLA FOR ALASKA               1163586.91   
1        C00792341        ALASKANS FOR NICK BEGICH                279455.12   
2        C00856716      NANCY DAHLSTROM FOR ALASKA                200826.90   
3        C00822163                  AM

In [12]:
# Create empty lists to store data
top_republicans = []

# Iterate through each seat
for seat in seat_data:
    df = seat_data[seat]
    
    # Get top Democrat
    dem_df = df[df['party_full'] == 'DEMOCRATIC PARTY']
    if not dem_df.empty:
        top_dem_receipts = dem_df['total_receipts'].max()
    else:
        top_dem_receipts = 0
        
    # Get top Republican
    rep_df = df[df['party_full'] == 'REPUBLICAN PARTY']
    if not rep_df.empty:
        top_rep = rep_df.loc[rep_df['total_receipts'].idxmax()]
        
        # Calculate distance from top Democrat
        distance = top_dem_receipts - top_rep['total_receipts']
        
        # Add to list with seat info
        top_rep_dict = top_rep.to_dict()
        top_rep_dict['state'] = seat[0]
        top_rep_dict['district'] = seat[1]
        top_rep_dict['distance_from_max'] = distance
        top_republicans.append(top_rep_dict)

# Create final dataframe
top_rep_df = pd.DataFrame(top_republicans)
top_rep_df



,candidate_election_year,candidate_id,candidate_name,candidate_pcc_id,candidate_pcc_name,cash_on_hand_end_period,committee_ids,coverage_end_date,incumbent_challenge_full,party_full,total_disbursements,total_receipts,state,district,distance_from_max
0,2024,H2AK01083,"BEGICH, NICHOLAS III",C00792341,ALASKANS FOR NICK BEGICH,279455.12,"[C00792341, C00856617]",2024-10-16,Challenger,REPUBLICAN PARTY,1801360.64,2039581.77,AK,00,9344110.14
1,2024,H4CO08034,"EVANS, TIMOTHY GABRIEL JOSEPH",C00849844,ELECTGABEEVANS.COM,363287.98,"[C00849844, C00856633]",2024-10-16,Challenger,REPUBLICAN PARTY,1934256.54,2297544.52,CO,08,5005594.08
2,2024,H4ME02291,"THERIAULT, AUSTIN LEO",C00852061,THERIAULT FOR CONGRESS,655051.13,[C00852061],2024-10-16,Challenger,REPUBLICAN PARTY,2554559.39,3209610.52,ME,02,3853372.85
3,2024,H2MI07123,"BARRETT, THOMAS MORE",C00793976,TOM BARRETT FOR CONGRESS,541319.88,"[C00793976, C00798686]",2024-10-16,Open seat,REPUBLICAN PARTY,4086079.83,4622110.60,MI,07,2423532.03
4,2024,H0MI08141,"JUNGE, PAUL",C00726687,PAUL JUNGE FOR CONGRESS,419157.93,"[C00726687, C00775304]",2024-10-16,Open seat,REPUBLICAN PARTY,4787401.45,5114278.16,MI,08,141436.81
5,2024,H4NC01137,"BUCKHOUT, LAURIE",C00853499,LAURIE BUCKHOUT FOR CONGRESS,557984.18,"[C00853499, C00857839]",2024-10-16,Challenger,REPUBLICAN PARTY,3721700.92,4279685.10,NC,01,1319806.62
6,2024,H8NM02156,"HERRELL, STELLA YVETTE",C00655571,YVETTE4CONGRESS,314872.34,[C00655571],2024-10-16,Challenger,REPUBLICAN PARTY,3750810.87,3958577.05,NM,02,2524971.34
7,2024,H8PA15195,"MACKENZIE, RYAN EDWARD",C00846501,MACKENZIE FOR CONGRESS COMMITTEE,278502.26,"[C00846501, C00856690, C00858357]",2024-10-16,Challenger,REPUBLICAN PARTY,1229150.88,1507653.14,PA,07,6653161.71
8,2024,H4PA08124,"BRESNAHAN, ROB",C00852137,ROB FOR PA,249018.30,[C00852137],2024-10-16,Challenger,REPUBLICAN PARTY,3875345.08,4124363.38,PA,08,3999779.61
9,2024,H2VA07170,"ANDERSON, DERRICK",C00791574,"DERRICK ANDERSON FOR VA, INC.",680644.04,[C00791574],2024-10-16,Open seat,REPUBLICAN PARTY,2174553.33,2851582.36,VA,07,12834457.99


In [13]:
# Sort the dataframe by distance_from_max in ascending order (lowest to highest)
top_rep_df = top_rep_df.sort_values('distance_from_max')
top_rep_df

,candidate_election_year,candidate_id,candidate_name,candidate_pcc_id,candidate_pcc_name,cash_on_hand_end_period,committee_ids,coverage_end_date,incumbent_challenge_full,party_full,total_disbursements,total_receipts,state,district,distance_from_max
17,2024,H0CA48198,"STEEL, MICHELLE",C00704981,MICHELLE STEEL FOR CONGRESS,2636113.06,[C00704981],2024-10-16,Incumbent,REPUBLICAN PARTY,7262323.59,9441899.44,CA,45,-4441011.11
19,2024,H2IA03119,"NUNN, ZACH",C00784389,IOWANS FOR ZACH NUNN,1036567.62,"[C00775122, C00784389]",2024-10-16,Incumbent,REPUBLICAN PARTY,4291597.19,5227097.15,IA,03,-400344.46
4,2024,H0MI08141,"JUNGE, PAUL",C00726687,PAUL JUNGE FOR CONGRESS,419157.93,"[C00726687, C00775304]",2024-10-16,Open seat,REPUBLICAN PARTY,4787401.45,5114278.16,MI,08,141436.81
23,2024,H2OR05209,"CHAVEZ-DEREMER, LORI",C00784520,LORI CHAVEZ-DEREMER FOR CONGRESS,613570.39,"[C00775767, C00784520]",2024-10-16,Incumbent,REPUBLICAN PARTY,5094032.87,5698141.84,OR,05,738634.63
20,2024,H6NE02125,"BACON, DONALD J",C00575167,DON BACON FOR CONGRESS,605114.81,[C00575167],2024-10-16,Incumbent,REPUBLICAN PARTY,5353940.90,5944820.21,NE,02,801945.64
18,2024,H8IA02043,"MILLER-MEEKS, MARIANNETTE JANE",C00558825,MILLER-MEEKS FOR CONGRESS,1471348.07,[C00558825],2024-10-16,Incumbent,REPUBLICAN PARTY,3707813.87,4850823.52,IA,01,999330.99
14,2024,H2CA20094,"VALADAO, DAVID",C00499392,VALADAO FOR CONGRESS,909774.29,[C00499392],2024-10-16,Incumbent,REPUBLICAN PARTY,3744004.38,4605340.10,CA,22,1167159.91
13,2024,H2CA13131,"DUARTE, JOHN",C00808279,JOHN DUARTE FOR CONGRESS,959391.99,"[C00798660, C00808279]",2024-10-16,Incumbent,REPUBLICAN PARTY,3213218.23,4147091.59,CA,13,1269170.42
5,2024,H4NC01137,"BUCKHOUT, LAURIE",C00853499,LAURIE BUCKHOUT FOR CONGRESS,557984.18,"[C00853499, C00857839]",2024-10-16,Challenger,REPUBLICAN PARTY,3721700.92,4279685.10,NC,01,1319806.62
12,2024,H2AZ02360,"CISCOMANI, JUAN",C00786194,CISCOMANI FOR CONGRESS,1224225.12,"[C00786194, C00798645]",2024-10-16,Incumbent,REPUBLICAN PARTY,5111408.21,6281197.73,AZ,06,1431864.67
